# Apple Classifier - Custom Dataset - 80% Accuracy

In this article, I will walk you through the apple classifier I created in using the fastai library with Google Colab. It's an incredibly noisy dataset and the total amount of training images is small but the model achieves 80% accuracy and makes mistakes in places where even humans would. For this reason, I think it's a great score. 

The classes of apple we are going to work with are: pink lady, braeburn, red delicious, golden delicious, granny smith and opal. The first three are red and the last three are green. 

First, we need to set up our coding environment

## Setting Up Google Colab

One of the best ways to build deep learning models is to use Google Colab as it's super easy to set up and gives you access to a GPU for free. 

First, you need to install the necessary packages to work with the fastai library.

```
!curl -s https://course.fast.ai/setup/colab | bash
Updating fastai...
Done.
```

Then mount your Google Drive as you will be storing and accessing all your data from there. Click the file tab on the left hand side, then 'Mount Drive' on the toolbar that appears and follow the instructions.

## Import Necessary Modules

We'll build this image classifier with the [fastai library](https://docs.fast.ai/), so we need to import everything from `fastai.vision` and also the `error_rate` which we'll print to the screen to see how well the model performs. 

```python
from fastai.vision import *
from fastai.metrics import error_rate
```

Now we're ready to start. 

## Setting Up Directory Structure

All the files and directories will revolved around the variable `path`, so let's create that now.

```python
from pathlib import Path

path = Path('data/apples')
```

There is no need to create any other path variables since you will just add one or two directories after `path` to get wherever you want to go.

Now we need to create 6 directories each named after a variety of apple.

```python
import os

# Create list of directory names
folders = ['pink_lady', 'braeburn', 'red_delicious', 
          'golden_delicious', 'granny_smith', 'opal']

# If the directory does not exist, create it
for f in folders:
    if not os.path.isdir(path/f):
        os.mkdir(path/f)
```

Perfect, now we are ready to start. Let's download our images.

## Downloading Your Custom Dataset

For each of the varieties of apple, go to Google Images and search `"pink lady apple"` including the double quotes as this will only search for images with an exact match. We're going to download a full page of 380 images, so scroll until you it says 'Show more results'.

To save us from manually downloading the images, we'll run some Javascript inside our browser. Press `CtrlShiftj` on Windows/Linux or `CmdOptj` on Mac to open the Javascript console and copy/paste the following.

```javascript
// Run this in your Javascript console
urls=Array.from(document.querySelectorAll('.rg_i')).map(el=> el.hasAttribute('data-src')?el.getAttribute('data-src'):el.getAttribute('data-iurl'));
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
```

It will download the links to all the image files on the page and save them in your Downloads folder as `download.csv`. Note: you may need to disable AdBlocker for this to work. 

Once you've downloaded the csv file, move it to `path` and rename it to `urls_pink_lady.csv` or something similarly appropriate. Do not move it inside the `pink_lady` folder as that is for image files only. 

This is a manual process and could be automated if you had tens or hundreds of categories but works fine for our purposes. 

Once you have downloaded the csv files for all 6 classes and moved them to `path`, it's time to actually download the images. 

```python
download_images(path/'urls_braeburn.csv', path/'braeburn')
```

You can either manually download them all or use a for loop like so

```python
url_files = [f'urls_{folder}.csv' for folder in folders]

# Sort lists to make sure iterating through them in the same order
files.sort(); folders.sort()

for url_file, folder in zip(url_files, folders):
    download_images(path/url_file, path/folder)
```

Note that your csv file may contain lots of blank lines and `download_images` will throw errors when trying to download them. It will still download all the lines with correct urls though, so just ignore the error messages. 

The final step is to verify the images and delete any that do not open.

```python
for f in folders:
    verify_images(path/f, delete=True, max_size=500)
```

## View Data

To look at our data, we need to get it into an `ImageDataBunch`. Note that I set the random seed beforehand so that the validation set is consistent every time I do it. 

```python
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train='.', valid_pct=0.2, 
                                  ds_tfms=get_transforms(), size=224, 
                                  num_workers=4).normalize(imagenet_stats)

```

I set aside 20% of the data to use for validation whilst transforming the images to ensure the model generalized well and finally normalizing them using the ImageNet statistics as we will be using a model that has been pretrained on ImageNet. 

Let's check the classes, c and the length of our training and validation sets

```python
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

(['braeburn',
 'golden_delicious',
 'granny_smith',
 'opal',
 'pink_lady',
 'red_delicious'],
 6,
 1460,
 365)
```
Looks good, as expected there are 6 classes, 1460 images in the training set and 365 in the validation set. Note that this is less than we initially downloaded due to corrupted files that could not be opened. 

Now let's look at a batch.

```python
data.show_batch(rows=4, figsize=(7,8))
```

<div>
    <img src="figures/initial_batch.png" align="left" width=400/>
</div>

Clearly this batch contains a lot of noise despite our best efforts when downloading the data. Let's build a quick model and see how well it performs regardless. Even if it performs terribly, we'll be able to use some great features of the fastai library to help with data cleaning down the road. 

## Training First Model

Initialize learner with the pretrained resnet34 model and fit for 4 epochs. 

```python
learn = cnn_learner(data, models.resnet34, metrics=error_rate)
learn.fit_one_cycle(4)
```

<div>
    <img src="figures/first_model.png" align="left" width=400/>
</div>

As expected, the model performs quite poorly on its first attempt. The error rate is 38.6% which means its accuracy is 61.4%. Let's save it regardless so that we can easily load this model again instead of training from scratch.

```python
learn.save('stage-1')
```

Now it's time to look at the results. I imagine a lot of these errors will come from noisy data. 

## Interpreting First Model

Initialize interpretation object and plot confusion matrix

```python
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
```

<div>
    <img src="figures/first_cm.png" align="left" width=400/>
</div>

The model seems to be good at classifying both Granny Smith and Red Delicious apples as they have the highest number of correct predictions by far.

It's also seems like it can distinguish between red and green apples quite well but finds it harder to distinguish between apples in their own category. If we look at the `golden_delicious` column, we see that it gets most confused between Opals and Granny Smith's rather than between itself and Pink Ladies and or Red Delicious. This is one of the main benefits of using a pretrained model, for one thing, it can already tell the difference between red and green.

Now let's look at one of the most helpful parts of the fastai library - seeing which images it was most wrong about. 

```python
interp.plot_top_losses(12)
```

The top losses are the images where the model was not confident with its prediction or where it was very confident and wrong. We are able to review and remove any incorrect images to improve our model rather than scanning through every single image that we've downloaded. 

<div>
    <img src="figures/first_losses.png" align="left" width=600/>
</div>

Clearly there are lots of images in our dataset that are not what we want! There are images with red and green apples in, the image on the far right middl is a metal container of some kind and the bottom right is a fruit salad.

Now we have to do some manual (boo!) data cleaning but unfortunately there is no other way to do it...


## Data Cleaning with Widgets

We're going to use the ImageCleaner widget from the fastai library. Unfortunately, this does not work in Google Colab, so we must switch to a Jupyter Notebook for it to run correctly. This is one reason why we save our models - they can be loaded into Jupyter to do this kind of cleaning.

```python
from fastai.widgets import *
```

## from_toplosses()

We'll first look at the images that had the highest losses as these are the most likely to be nosiy/unwanted. 

Let's create a new databunch containing all of our images including our validation set. 

```python
db = (ImageList.from_folder(path)
                 .split_none()
                 .label_from_folder()
                 .transform(get_transforms(), size=224)
                 .databunch()
     )
```

Now we load in our learner.

```python
learn_clean = cnn_learner(db, models.resnet34, metrics=error_rate)
learn_clean.load('stage-1')
```

And create a `DatasetFormatter` object which lets us modify our dataset.

```python
ds, idxs = DatasetFormatter.from_toplosses(learn_clean)
```

We store the dataset (i.e. all the image files) and the indexes of them in `ds` and `idxs` respectively. Because we used `from_toplosses()` the two objects are both in descending order of losses starting with the highest loss. 

```python
ImageCleaner(ds, idxs, path)
```

Finally, pass them to `ImageCleaner` to open up the widget. You can choose to relabel certain images or delete them. This will not actually delete them from the system but store them in `cleaned.csv` which you can then use to create a new databunch and build a new model with. Regardless of what your current working directory is, `cleaned.csv` is stored in `path`.

Go through the images until you do 2-3 passes of images that don't need to be modified. 

Great! We've now removed loads of nosiy/unwanted images. Now let's do the same process to remove duplicate images. 

## from_similars()

The `from_similars()` method on `DatasetFormatter` will return the entire dataset but ordered in descending order with images that are the most similar first. So, you'll be able to find duplicate/identical images very quickly and remove them. 

The following code is (appropriately) similar to that above, so it is just presented with comments. 

```python
# Create new databunch with cleaned.csv
db = (ImageList.from_csv(path, 'cleaned.csv', folder='.')
                 .split_none()
                 .label_from_df()
                 .transform(get_transforms(), size=224)
                 .databunch()
     )

# Initialize new learner and load pre-trained model
learn_clean = cnn_learner(db, models.resnet34)
learn_clean.load('stage-1')

# Use .from_similars()
ds, idxs = DatasetFormatter.from_similars(learn_clean)

# Start ImageCleaner to look for duplicates
ImageCleaner(ds, idxs, path, duplicates=True)
```

Go through the same process until you are happy you've removed a large number of duplicate images. The `cleaned.csv` file is automatically updated on each iteration.

Now that the data has been cleaned, it's time to re-train the model.

## Training Cleaned Model

First we need to create a databunch from `cleaned.csv`. 

```python
# Same random seed as before
np.random.seed(42)

# from_csv rather than from_folder
data = ImageDataBunch.from_csv(path, folder='.', valid_pct=0.2,
                               csv_labels='cleaned.csv', ds_tfms=get_transforms(),
                               size=224, num_workers=4).normalize(imagenet_stats)
```

In the first iteration, I removed 22% of the data, let's see what accuracy gains that gives us.

```python
learn = cnn_learner(data, models.resnet34, metrics=error_rate)
learn.fit_one_cycle(4)
```

<div>
    <img src="figures/second_model.png" align="left" width=400/>
</div>

Accuracy increased from 61.3% to 72.3% which is a 17.9% improvement on the initial model. This is a huge step forwards.

```python
learn.save('stage-2')
```

Up to now, we've just been training the last few layers of resnet34. Since we've now got a decent level of accuracy, let's train the entire model by first unfreezing the layers and using the learning rate finder and plotting its results. 

```python
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()
```

<div>
    <img src="figures/lr_finder.png" align="left" width=400/>
</div>

We want to find the steepest part of the slope which looks like it starts at around `2e-5`, so let's run the model a few more times using a slice of learning rates.

```python
learn.fit_one_cycle(2, max_lr=slice(2e-5, 1e-3))
```

<div>
    <img src="figures/third_model.png" align="left" width=400/>
</div>

Great, the accuracy is now up to 77.9%.

Let's look at the confusion matrix and top losses to see if the incorrect predictions are caused by noisy data. 

```python
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
```

<div>
    <img src="figures/second_cm.png" align="left" width=400/>
</div>

The model seems to be great at distinguishing between most of the varieties of apple apart from Pink Lady. That category has, by far, the highest number of incorrect predicitions. 

Let's look at the top losses. 

```python
interp.plot_top_losses(20)
```

<div>
    <img src="figures/second_losses.png" align="left" width=600/>
</div>

Looking at the losses a few things emerge:
1. Some images clearly should not be there e.g. bottles of brandy and some that contain both red and green apples
2. Apples are very similar 

As someone who has mainly eaten braeburn and pink lady apples for most of his life, when I look at some of the images of pink ladies, I think they are braeburns and vice versa.

<div>
    <img src="figures/pinkburn.png" align="left" width=400/>
</div>

If you were to show me the above image, I would say it was a Pink Lady with a Braeburn sticker on it. This is great because it means are model is making mistakes that even humans would make. 

## Fine Tuning

Now that we know the resnet34 model is working well, it is time to 
1. Do further data cleaning by removing more incorrect images, and
2. Train a full resnet50 model

These steps are pretty much identical to the ones I did above, so, for brevity, I will not show the code. The only real difference is that we use `models.resnet50` when creating `learn` instead of `models.resnet34`. 

<div>
    <img src="figures/final_model.png" align="left" width=400/>
</div>

After repeating the above process and training the entire resnet50 model, we achieved an error rate of 19.0%, or an accuracy of 81.0%! Not bad for a model created from a few hundred images taken from Google that started with an accuracy of 61.4%. 

Of course, further improvements could be made by getting more data, looking at weight decay and data augmentation but for now, I think 81% accuracy is pretty good, especially given that it is making mistakes that humans (I) would also make. 